## Homework: Agents

In this homework, we will learn more about function calling,
and we will also explore MCP - model-context protocol. 


## Preparation

First, we'll define a function that we will use when building
our agent. 

It will generate fake weather data:

In [1]:
import random

known_weather_data = {
    'berlin': 20.0
}

def get_weather(city: str) -> float:
    city = city.strip().lower()

    if city in known_weather_data:
        return known_weather_data[city]

    return round(random.uniform(-5, 35), 1)

## Q1. Define function description

We want to use it as a tool for our agent, so we need to 
describe it 

How should the description for this function look like? Fill in missing parts

In [51]:
get_weather_tool = {
    "type": "function",
    "name": "get_weather",
    "description": "Get randomly generated weather data from a city",
    "parameters": {
        "type": "object",
        "properties": {
            "city": {
                "type": "string",
                "description": "Get a weather data from given city"
            }
        },
        "required": ["city"],
        "additionalProperties": False
    }
}

!wget https://raw.githubusercontent.com/alexeygrigorev/rag-agents-workshop/refs/heads/main/chat_assistant.py

In [20]:
import chat_assistant

In [21]:
from openai import OpenAI
client = OpenAI()

In [ ]:
tools = chat_assistant.Tools()

In [55]:

tools.add_tool(get_weather, get_weather_tool)
tools.get_tools()

[{'type': 'function',
  'name': 'get_weather',
  'description': 'Get randomly generated weather data from a city',
  'parameters': {'type': 'object',
   'properties': {'city': {'type': 'string',
     'description': 'Get a weather data from given city'}},
   'required': ['city'],
   'additionalProperties': False}}]

In [60]:
developer_prompt = """
You are an answering machine
You're given a question by a human and your task is to answer it
"""
chat_interface = chat_assistant.ChatInterface()


In [ ]:
chat = chat_assistant.ChatAssistant(
    tools=tools,
    developer_prompt=developer_prompt,
    chat_interface=chat_interface,
    client=client
)

In [ ]:
chat.run()


Chat ended.


# Q2. Adding another tool

Let's add another tool - a function that can add weather data
to our database:

```python
def set_weather(city: str, temp: float) -> None:
    city = city.strip().lower()
    known_weather_data[city] = temp
    return 'OK'
```

Now let's write a description for it.

What did you write?

Optionally, you can test it after adding this function.

In [47]:
def set_weather(city: str, temp: float) -> None:
    city = city.strip().lower()
    known_weather_data[city] = temp
    return 'OK'

In [63]:
set_weather_tool = {
    "type": "function",
    "name": "set_weather",
    "description": "add weather data entry into known_weather_data",
    "parameters": {
        "type": "object",
        "properties": {
            "city": {
                "type": "string",
                "description": "set the city name "
            },
            "temp": {
                "type": "number",
                "description": "Set the temperature"
            }
        },
        "required": ["city","temp"],
        "additionalProperties": False
    }
}

In [66]:

tools.add_tool(set_weather,set_weather_tool)
tools.get_tools()

[{'type': 'function',
  'name': 'get_weather',
  'description': 'Get randomly generated weather data from a city',
  'parameters': {'type': 'object',
   'properties': {'city': {'type': 'string',
     'description': 'Get a weather data from given city'}},
   'required': ['city'],
   'additionalProperties': False}},
 {'type': 'function',
  'name': 'set_weather',
  'description': 'add weather data entry into known_weather_data',
  'parameters': {'type': 'object',
   'properties': {'city': {'type': 'string',
     'description': 'set the city name '},
    'temp': {'type': 'number', 'description': 'Set the temperature'}},
   'required': ['city', 'temp'],
   'additionalProperties': False}}]

In [67]:
chat = chat_assistant.ChatAssistant(
    tools=tools,
    developer_prompt=developer_prompt,
    chat_interface=chat_interface,
    client=client
)

In [73]:
chat.run()

Chat ended.


In [74]:
print(known_weather_data)

{'berlin': 20.0, 'jakarta': 29.7, 'melbourne': 6}


## MCP

MCP stands for Model-Context Protocol. It allows LLMs communicate
with different tools (like Qdrant). It's function calling, but 
one step further:

* A tool can export a list of functions it has
* When we include the tool to our Agent, we just need to include the link to the MCP server

## Q3. Install FastMCP

Let's install a library for MCP - [FastMCP](https://github.com/jlowin/fastmcp):

```bash
pip install fastmcp
```

What's the version of FastMCP you installed?

In [82]:
!uv pip install fastmcp

/opt/homebrew/Cellar/python@3.13/3.13.5/Frameworks/Python.framework/Versions/3.13/lib/python3.13/pty.py:95: DeprecationWarning: This process (pid=6272) is multi-threaded, use of forkpty() may lead to deadlocks in the child.
  pid, fd = os.forkpty()


Using Python 3.13.5 environment at: /Users/robertusprimusanto/Documents/code/dataTalksClub/llmzoomcamp-202506/homework-llmzoomcamp-2025/.venv
Resolved 44 packages in 2.18s                                        
⠙ Preparing packages... (0/1)                                                   
⠙ Preparing packages... (0/1)--------------     0 B/196.56 KiB          
⠙ Preparing packages... (0/1)-------------- 16.00 KiB/196.56 KiB        
⠙ Preparing packages... (0/1)-------------- 32.00 KiB/196.56 KiB        
⠙ Preparing packages... (0/1)-------------- 48.00 KiB/196.56 KiB        
⠙ Preparing packages... (0/1)-------------- 62.90 KiB/196.56 KiB        
⠙ Preparing packages... (0/1)-------------- 78.90 KiB/196.56 KiB        
⠙ Preparing packages... (0/1)-------------- 94.90 KiB/196.56 KiB        
⠙ Preparing packages... (0/1)m------------- 110.90 KiB/196.56 KiB       
⠙ Preparing packages... (0/1)---------- 126.90 KiB/196.56 KiB       
⠙ Preparing packages... (0/1)---------- 142.90 KiB/196

In [83]:
!fastmcp version

/opt/homebrew/Cellar/python@3.13/3.13.5/Frameworks/Python.framework/Versions/3.13/lib/python3.13/pty.py:95: DeprecationWarning: This process (pid=6272) is multi-threaded, use of forkpty() may lead to deadlocks in the child.
  pid, fd = os.forkpty()


FastMCP version:                                                          2.10.5
MCP version:                                                              1.11.0
Python version:                                                           3.13.5
Platform:                                      macOS-15.5-arm64-arm-64bit-Mach-O
FastMCP root path: /Users/robertusprimusanto/Documents/code/dataTalksClub/llmzo…


Q4. Simple MCP Server